In [ ]:
from numba import jit,prange,cuda,vectorize,float32,float64 #牢记prange也要导入
import numpy as np
import math
import cupy

In [1]:
@jit(nopython=True,parallel=True)
def model_initial_state(MatA,MU,SIGMA):
    row,col=MatA.shape
    for i in prange(row):
        for j in prange(col):
            RN=always_large_0(MU,SIGMA)
            MatA[i,j]=RN
    return MatA
    
@jit(nopython=True)
def always_large_0(MU,SIGMA):
    a=np.random.normal(MU,SIGMA)
    while a<0:
        a=np.random.normal(MU,SIGMA)
    return a

@jit(nopython=True) 
def model_initial_evolution(MatB): 
    row,col=MatB.shape
    MatA=np.empty((row+2,col+2),dtype=np.float32)
    MatA[1:row+1,1:col+1]=MatB
    return MatA

@jit(nopython=True)
def model_evolution_update(MatA): 
    row,col=MatA.shape
    row=row-2
    col=col-2
    MatB=np.empty((row,col),dtype=np.float32)
    MatB=MatA[1:row+1,1:col+1]
    return MatB

@jit(nopython=True) 
def periodic_boundary_condition(MatA,MatB):
    row,col=MatB.shape
    MatA[0,1:col+1]=MatB[row-1,:]
    MatA[row+1,1:col+1]=MatB[0,:] 
    MatA[1:row+1,0]=MatB[:,col-1] #Here a big mistake happened:1:row+1 -> 1:col+1
    MatA[1:row+1,col+1]=MatB[:,0]
    MatA[0,0]=MatB[row-1,col-1]
    MatA[0,col+1]=MatB[row-1,0]
    MatA[row+1,0]=MatB[0,col-1]
    MatA[row+1,col+1]=MatB[0,0]
    return MatA 

@cuda.jit
def kernel_compute(MatA,MatB,par1,par2,par3,par4,MatC): 
    x,y=cuda.grid(2)                               
    if (x>0) and (x<MatA.shape[0]-1) and (y>0) and (y<MatA.shape[1]-1): 
        GlobalChange=MatA[x,y]*math.exp(par1*(1-MatA[x,y]))*(1+par2*MatB[x,y])
        NeighbourChange=MatA[x-1,y]*math.exp(par1*(1-MatA[x-1,y]))*(1+par2*MatB[x-1,y])+\
        MatA[x+1,y]*math.exp(par1*(1-MatA[x+1,y]))*(1+par2*MatB[x+1,y])+\
        MatA[x,y+1]*math.exp(par1*(1-MatA[x,y+1]))*(1+par2*MatB[x,y+1])+\
        MatA[x,y-1]*math.exp(par1*(1-MatA[x,y-1]))*(1+par2*MatB[x,y-1])
        MatC[x,y]=GlobalChange*par3+NeighbourChange*par4
    else:
        MatC[x,y]=MatA[x,y]

@cuda.jit
def kernel_pbc(MatA): 
    x,y=cuda.grid(2)
    row1,col1=MatA.shape
    if (x==0) and (y>0) and (y<col1-1):
        MatA[x,y]=MatA[row1-2,y]
    elif (x==row1-1) and (y>0) and (y<col1-1):
        MatA[x,y]=MatA[1,y]
    elif (y==0) and (x>0) and (x<row1-1):
        MatA[x,y]=MatA[x,col1-2]
    elif (y==col1-1) and (x>0) and (x<row1-1):
        MatA[x,y]=MatA[x,1]
    elif (x==0) and (y==0):
        MatA[x,y]=MatA[row1-2,col1-2]
    elif (x==0) and (y==col1-1):
        MatA[x,y]=MatA[row1-2,1]
    elif (x==row1-1) and (y==0):
        MatA[x,y]=MatA[1,col1-2]
    elif (x==row1-1) and (y==col1-1):
        MatA[x,y]=MatA[1,1]
    else:
        MatA[x,y]=MatA[x,y]

@cuda.jit
def kernel_ass(MatA,MatB):
    x,y=cuda.grid(2)
    MatA[x,y]=MatB[x,y]

@cuda.jit
def kernel_separate_compute(MatA,MatB,par1,par2,par3,par4,MatC): 
    x,y=cuda.grid(2)                            
    if (x>0) and (x<MatA.shape[0]-1) and (y>0) and (y<MatA.shape[1]-1):
        GlobalChange=device_Global(MatA,MatB,par1,par2)
        NeighbourChange=device_Neighbour(MatA,MatB,par1,par2)
        MatC[x,y]=GlobalChange*par3+NeighbourChange*par4
    else:
        MatC[x,y]=MatA[x,y]
        
@cuda.jit(device=True)
def device_Global(MatA,MatB,par1,par2):
    x,y=cuda.grid(2)
    GlobalChange1=MatA[x,y]*math.exp(par1*(1-MatA[x,y]))*(1+par2*MatB[x,y])
    return GlobalChange1

@cuda.jit(device=True)
def device_Neighbour(MatA,MatB,par1,par2):
    x,y=cuda.grid(2)
    NeighbourChange1=MatA[x-1,y]*math.exp(par1*(1-MatA[x-1,y]))*(1+par2*MatB[x-1,y])+\
    MatA[x+1,y]*math.exp(par1*(1-MatA[x+1,y]))*(1+par2*MatB[x+1,y])+\
    MatA[x,y+1]*math.exp(par1*(1-MatA[x,y+1]))*(1+par2*MatB[x,y+1])+\
    MatA[x,y-1]*math.exp(par1*(1-MatA[x,y-1]))*(1+par2*MatB[x,y-1])
    return NeighbourChange1

In [2]:
def Ricker2D(L,B,cB):
    Burntime=int(1e5)
    T=int(1e4)  
#    L=int(512)
    M=L 
    N=L 
    LF=0.1
    z=4.0
    r=2.3
#    B=0.12
    GF=1-LF
    LFR=LF/z
    c0=-1 #pre_model_state count
    c1=-1 #post_model_state count
    modelstate=np.empty((M-2,N-2),dtype=np.float32)
    MU=np.float32(0.5)
    SIGMA=np.float32(0.1)
    modelstate=model_initial_state(modelstate,MU,SIGMA)
    Modelevolution=model_initial_evolution(modelstate)
    Modelevolution=periodic_boundary_condition(Modelevolution,modelstate)
    Noise_device=cupy.empty(shape=(M,N),dtype=np.float32)
    Modeltemp_device=cupy.empty(shape=(M,N),dtype=np.float32)
    Modelevolution_device=cupy.asarray(Modelevolution)  
    threadsperblock=(32,32) 
    blockspergrid_x=math.ceil(Modelevolution.shape[0]/threadsperblock[0])
    blockspergrid_y=math.ceil(Modelevolution.shape[1]/threadsperblock[1])
    blockspergrid=(blockspergrid_x,blockspergrid_y)
    for k in range(Burntime+T+1):
        Noise_device=cupy.random.randn(M,N,dtype=np.float32)
        kernel_pbc[blockspergrid,threadsperblock](Noise_device)
        kernel_separate_compute[blockspergrid,threadsperblock](Modelevolution_device,Noise_device,r,B,GF,LFR,Modeltemp_device)
        kernel_ass[blockspergrid,threadsperblock](Modelevolution_device,Modeltemp_device)
        kernel_pbc[blockspergrid,threadsperblock](Modelevolution_device)
        if (k>=Burntime-1) and ((k-Burntime+1)%200==0):
            Modelevolution=cupy.asnumpy(Modelevolution_device)
            c0+=1
            path='D:\\ZhenpengGe\\Ricker2D_L4096\\'
            filename0=str(cB).zfill(4)+'_'+str(c0).zfill(5)+'_0.bin' #cB means the No. of cases, c0 means the No. of samples，0 means pre-state, 1 means post-state
            Modelevolution.tofile(path+filename0) 
        elif (k>Burntime-1) and ((k-Burntime+1)%200==1):
            Modelevolution=cupy.asnumpy(Modelevolution_device)
            c1+=1
            path='D:\\ZhenpengGe\\Ricker2D_L4096\\'
            filename1=str(cB).zfill(4)+'_'+str(c1).zfill(5)+'_1.bin'
            Modelevolution.tofile(path+filename1) 

**Run model**

In [ ]:
L=int(4096) 
StartB=np.float32(0.135) 
EndB=np.float32(0.165) 
Loop=np.linspace(StartB,EndB,num=144,dtype=np.float32)
#RealLoop=np.concatenate((Loop[0:17],np.linspace(Loop[18],Loop[28],num=23,dtype=np.float32),Loop[29:63]))
cB=-1
for mB in Loop: 
    cB+=1 
    Ricker2D(L,mB,cB)

In [3]:
L=int(4096) 
StartB=np.float32(0.135) 
EndB=np.float32(0.15) 
Loop=np.linspace(StartB,EndB,num=72,dtype=np.float32)
#Loop=np.array([0.135,0.15])
#RealLoop=np.concatenate((Loop[0:17],np.linspace(Loop[18],Loop[28],num=23,dtype=np.float32),Loop[29:63]))
cB=-1
for mB in Loop:
    cB+=1 
    Ricker2D(L,mB,cB)
#    print(cB)

In [ ]:
L=int(4096) 
StartB=np.float32(0.135) 
EndB=np.float32(0.165) 
TLoop=np.linspace(StartB,EndB,num=144,dtype=np.float32)
Loop=TLoop[72:]
cB=71
for mB in Loop:
    cB+=1 
    Ricker2D(L,mB,cB)